In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn
from scipy import stats
import missingno as msno
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
train_df=pd.read_csv('../input/bike-sharing-demand/train.csv')
test_df=pd.read_csv('../input/bike-sharing-demand/test.csv')

## EDA & Feature Engineering

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
train_df.isnull().sum() 

In [ ]:
train_df.duplicated().sum()


In [ ]:
train_df.drop_duplicates(inplace = True)


In [ ]:
test_df.duplicated().sum()


In [ ]:
train_df.groupby('season')['count'].hist(alpha = 0.4)
plt.legend( ['spring','summer','fall','winter'])

In [ ]:
train_df.groupby('season').count()

In [ ]:
plt.figure(figsize=(16,8))
sns.histplot(train_df['count'],kde = True)
plt.show()

In [ ]:
sns.catplot(x='weather',data=train_df,kind='count',height=5,aspect=1.5)


In [ ]:
cor_mat= train_df[:].corr()
mask = np.array(cor_mat)
mask[np.tril_indices_from(mask)] = False
fig=plt.gcf()
fig.set_size_inches(30,12)
sns.heatmap(data=cor_mat,mask=mask,square=True,annot=True,cbar=True)

In [ ]:
train_df["hour"] = [t.hour for t in pd.DatetimeIndex(train_df.datetime)]
train_df["day"] = [t.dayofweek for t in pd.DatetimeIndex(train_df.datetime)]
train_df["month"] = [t.month for t in pd.DatetimeIndex(train_df.datetime)]
train_df['year'] = [t.year for t in pd.DatetimeIndex(train_df.datetime)]

train_df['year'] = train_df['year'].map({2011:0, 2012:1})
train_df.head()

In [ ]:
test_df["hour"] = [t.hour for t in pd.DatetimeIndex(test_df.datetime)]
test_df["day"] = [t.dayofweek for t in pd.DatetimeIndex(test_df.datetime)]
test_df["month"] = [t.month for t in pd.DatetimeIndex(test_df.datetime)]
test_df['year'] = [t.year for t in pd.DatetimeIndex(test_df.datetime)]
test_df['year'] = test_df['year'].map({2011:0, 2012:1})
test_df.head()

In [ ]:
train_df.drop(['datetime','casual','registered','atemp'],axis=1,inplace=True)


In [ ]:
date_time = test_df.datetime
test_df.drop(['datetime','atemp'],axis=1,inplace=True)


In [ ]:
train_df.head()

### Model

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

In [ ]:
train_df['count']=np.log(train_df['count']+1)
Y_train = train_df['count']
X_train = train_df.drop(['count'], axis=1)
# X_train['Date']=X_train['Date'].map(dt.datetime.toordinal)
x_train, x_test, y_train, y_test = train_test_split(X_train , Y_train,  train_size=0.8, test_size=0.2, random_state =0)

In [ ]:
import xgboost as xgb

xg_reg = xgb.XGBRegressor( colsample_bytree = 0.8, learning_rate = 0.149,min_child_weight =25, max_depth = 3,  n_estimators = 450)
xg_reg.fit(x_train,y_train)
pred=xg_reg.predict(x_test)
sc=np.sqrt(mean_squared_error(y_train , xg_reg.predict(x_train)))
score=np.sqrt(mean_squared_error(y_test, pred))
print("The score is:",score)
print("The score is:",sc)


In [ ]:
final_y_test = xg_reg.predict(test_df) 
final_y_test = np.round(np.exp(final_y_test)-1)

In [ ]:
output = pd.DataFrame({'datetime': date_time,
                       'count': final_y_test})
output.to_csv('submission.csv', index=False)

In [ ]:
output